In [ ]:
import pandas as pd
import numpy as np
import re
import pdfplumber
import camelot

In [109]:
def extract_pdf_tables(file: str, password: str = None, pages: str = '1', export=False, output_type: str = 'csv',
                       output: str = "extraction_output", combine: bool = False):
    import camelot
    import os
    from zipfile import ZipFile
    # camelot-py 0.10.1 documentation: https://buildmedia.readthedocs.org/media/pdf/camelot-py/latest/camelot-py.pdf
    # read tables in pdf
    tables = camelot.read_pdf(file, password=password, pages=pages)

    # specify output type in a dict
    output_type = output_type.lower()
    ext_dict = {"csv": "csv",
                "excel": "xlsx",
                "html": "html",
                "json": "json",
                "markdown": "md",
                "sqlite": "db"
                }
    # create output file name
    output_name = ".".join([output, ext_dict[output_type]])
    if export:
        tables.export(output_name, f=output_type, compress=True)  # will generate a zip containing all results

        # decompress zip to generate a folder containing all results
        zip_file = ".".join([output, "zip"])
        with ZipFile(zip_file, "r") as zip:
            zip.extractall(os.path.join(os.path.abspath('.'), output))

    if combine:
        # create a list to store all tables extracted and combine them (output will be the combined table)
        tables_list = [tb.df for tb in tables]
        table_combined = pd.concat(tables_list, ignore_index=True)
        # table_combined.to_csv(".".join([output + '_all', 'csv']), index=False, header=True)
        return table_combined

    # output is a table list object, unless combine=T
    return tables

In [165]:
pdf_file = "hdb_ec_land_sales.pdf"
summary_pages = 6
name_parser = pdf_file.split('_')
data_source = name_parser[0].lower()
devt_type = name_parser[1].lower()

with pdfplumber.open(pdf_file) as pdf:
    total_pages = len(pdf.pages)
    land_parcel_list = []
    page_list = []
    for page in range(summary_pages, total_pages):
        pdf_page = pdf.pages[page]
        try:
            raw_text = pdf_page.extract_text()
            cleaned_text = re.sub(r'\n+', ' ', raw_text)
            cleaned_text = re.sub(r' +', ' ', cleaned_text).strip()
            if 'hdb_mixed' in pdf_file:
                land_parcel = re.findall(r'Land *Parcel ?:? *(.*?) *?S/N', cleaned_text)[0]
            elif 'hdb_condo' in pdf_file or 'hdb_ec' in pdf_file:
                land_parcel = re.findall(r'Land *Parcel *:? *(.*?) *Tender *Bid?', cleaned_text)[0]
            land_parcel_list.append(land_parcel)
            # print(land_parcel)
        except:
            land_parcel_list.append(np.nan)

        page_list.append(page+1)

In [166]:
df_list = []
for i in range(len(page_list)):
    try:
        table = extract_pdf_tables(pdf_file, pages=str(page_list[i]), output_type='csv', output='tenderer_details_hdb_mixed')[0].df
        table.columns = [str(x).lower() for x in table.iloc[0, :].values]
        # cleaned_header = []
        # for col in list(table.columns):
        #     try:
        #         col_new = re.sub(r'\\*', '', re.sub(r'\\n*', '', col)).strip()
        #         cleaned_header.append(re.sub(r' +', ' ', col_new))
        #     except:
        #         cleaned_header.append(col)
        table = table.drop(0, axis=0).rename(columns={table.columns[0]: "tenderer_rank", table.columns[1]: "tenderer_name", table.columns[2]: "tender_price"})
        table["land_parcel"] = land_parcel_list[i]
        table["source"] = data_source
        table["devt_type"] = devt_type
        df_list.append(table)

    except:
        print(f'Error in extracting tender details for {land_parcel_list[i]}')

df_list

[   tenderer_rank                                      tenderer_name  \
 1              1                                   Pidemco Land Ltd   
 2              2  NTUC Choice Homes Co-operative Ltd / Koh \nBro...   
 3              3                            Hong Leong Holdings Ltd   
 4              4                                 DBS Realty Pte Ltd   
 5              5                         Centrepoint Properties Ltd   
 6              6                             Econ International Ltd   
 7              7  Yeo Hiap Seng Ltd / LKN Construction PL / \nNa...   
 8              8  Hong Bee Investment PL/ Consolidated Land \nIn...   
 9              9      Allgreen Properties Ltd / Hoe Seng Co Pte Ltd   
 10            10        Singapore Technologies Construction Pte Ltd   
 11            11                Shun Shing Construction (S) Pte Ltd   
 12            12                 Crystal Heights Investment Pte Ltd   
 13            13                         Winavic Investment Pte

In [168]:
merged_df3 = pd.concat(df_list, ignore_index=True)
merged_df3["devt_type"] = 'residential'
merged_df3

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
0,1,Pidemco Land Ltd,"196,888,000.00",Woodlands E1,hdb,residential
1,2,NTUC Choice Homes Co-operative Ltd / Koh \nBro...,"193,900,000.00",Woodlands E1,hdb,residential
2,3,Hong Leong Holdings Ltd,"188,000,000.00",Woodlands E1,hdb,residential
3,4,DBS Realty Pte Ltd,"183,300,000.00",Woodlands E1,hdb,residential
4,5,Centrepoint Properties Ltd,"179,300,000.00",Woodlands E1,hdb,residential
...,...,...,...,...,...,...
481,9,JBE (S) Pte. Ltd.,"211,000,000.00",Bukit Batok E11,hdb,residential
482,1,CDL Zenith Pte. Ltd.,"336,068,000.00",Bukit Batok E12,hdb,residential
483,2,Sim Lian Land Pte Ltd & Sim Lian Development \...,"335,508,000.00",Bukit Batok E12,hdb,residential
484,3,Intrepid Investments Pte. Ltd. and TID \nResid...,"332,526,222.00",Bukit Batok E12,hdb,residential


In [164]:
merged_df2 = pd.concat(df_list, ignore_index=True)
merged_df2["devt_type"] = 'residential'
merged_df2

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
0,1,First Capital Corporation Ltd,"29,260,000.00",Bedok S1,hdb,residential
1,2,Goldvein Pte Ltd,"19,000,000.00",Bedok S1,hdb,residential
2,3,Ladyhill (Pte) Ltd,"16,800,000.00",Bedok S1,hdb,residential
3,4,Allgreen Properties Ltd,"13,880,000.00",Bedok S1,hdb,residential
4,1,First Capital Corporation Ltd,"25,270,000.00",Bukit Merah S1,hdb,residential
...,...,...,...,...,...,...
859,5,"UOL Venture Investments Pte. Ltd., Singland \n...","417,280,000.00",Queenstown S9b,hdb,residential
860,6,Excelsior Investments Pte Ltd,"402,080,000.00",Queenstown S9b,hdb,residential
861,7,Sing Holdings Limited,"389,000,000.00",Queenstown S9b,hdb,residential
862,8,MCL Land (Regency) Pte. Ltd.,"387,000,000.00",Queenstown S9b,hdb,residential


In [108]:
merged_df = pd.concat(df_list, ignore_index=True)
merged_df.land_parcel.unique()

array(['Hougang N5NC', 'Hougang N9NC', 'Woodlands N4NC', 'Sembawang P1',
       'Sengkang N1NC', 'Sengkang P1', 'Jurong West P3',
       'Choa Chu Kang P2', 'Bedok P1', 'Punggol P1', 'Yishun P4',
       'Yishun P3', 'Yishun P5', 'Bidadari P1', 'Pasir Ris P2'],
      dtype=object)

In [254]:
final_df = pd.concat([final_df, merged_df3], ignore_index=True)

In [255]:
final_df.to_csv('cache.csv', index=False)

In [272]:
final_df = pd.read_csv('cache.csv')
final_df.isna().sum()

tenderer_rank     2
tenderer_name    10
tender_price      2
land_parcel       0
source            0
devt_type         0
dtype: int64

### deal with 1st col issue

In [273]:
no_name = final_df[final_df.tenderer_name.isna()]
no_name

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
2000,3A MCL Land Ltd,NaN,"88,000,000.00",Choa Chu Kang E3,hdb,residential
2089,11 Wee Hur Development Pte Ltd,NaN,"90,000,000.00",Pasir Ris E9,hdb,residential
2153,10 Mezzo Development Pte Ltd,NaN,"169,000,000.00",Pasir Ris E10,hdb,residential
2159,5B Mezzo Development Pte Ltd,NaN,"141,000,000.00",Punggol E12,hdb,residential
2221,10 Wee Hur Development Pte Ltd,NaN,"132,630,000.00",Jurong West E3,hdb,residential
2300,10a Centra One Pte. Ltd.,NaN,"108,000,000.00",Choa Chu Kang E12,hdb,residential
2301,10b Sim Lian Land Pte Ltd,NaN,"108,000,000.00",Choa Chu Kang E12,hdb,residential
2322,11 Greatview Investment Pte Ltd,NaN,"200,780,000.00",Sengkang E20,hdb,residential
2338,11 GLL C Pte. Ltd.,NaN,"397,533,000.00",Punggol E13,hdb,residential
2343,16 MCL Land (Edge) Pte. Ltd.,NaN,"382,000,000.00",Punggol E13,hdb,residential


In [274]:
split_df = no_name.tenderer_rank.apply(lambda x: re.sub(r' +', ' ', x).strip()).apply(lambda x: x.split(' '))
err_idx = split_df.index
final_df.tenderer_rank[err_idx]

2000                    3A  MCL Land Ltd
2089     11  Wee Hur Development Pte Ltd
2153       10  Mezzo Development Pte Ltd
2159       5B  Mezzo Development Pte Ltd
2221     10  Wee Hur Development Pte Ltd
2300           10a  Centra One Pte. Ltd.
2301          10b  Sim Lian Land Pte Ltd
2322    11  Greatview Investment Pte Ltd
2338                 11  GLL C Pte. Ltd.
2343       16  MCL Land (Edge) Pte. Ltd.
Name: tenderer_rank, dtype: object

In [275]:
ranking = split_df.apply(lambda x: x[0])
ranking

2000     3A
2089     11
2153     10
2159     5B
2221     10
2300    10a
2301    10b
2322     11
2338     11
2343     16
Name: tenderer_rank, dtype: object

In [276]:
names = split_df.apply(lambda x: ' '.join(x[1:]) if len(x)>1 else np.nan)
names

2000                    MCL Land Ltd
2089     Wee Hur Development Pte Ltd
2153       Mezzo Development Pte Ltd
2159       Mezzo Development Pte Ltd
2221     Wee Hur Development Pte Ltd
2300            Centra One Pte. Ltd.
2301           Sim Lian Land Pte Ltd
2322    Greatview Investment Pte Ltd
2338                 GLL C Pte. Ltd.
2343       MCL Land (Edge) Pte. Ltd.
Name: tenderer_rank, dtype: object

In [277]:
final_df.tenderer_name[err_idx] = names
final_df.tenderer_rank[err_idx] = ranking
final_df.isna().sum()

tenderer_rank    2
tenderer_name    0
tender_price     2
land_parcel      0
source           0
devt_type        0
dtype: int64

### deal with 2nd col issues

In [278]:
no_rank = final_df[final_df.tenderer_rank.isna()]
no_rank

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
2163,NaN,3A Master Contract Services Pte Ltd & Keong \...,"208,500,000.00",Sembawang E1a,hdb,residential
2241,NaN,12 Wee Hur Development Pte Ltd,"139,000,000.00",Sengkang E4,hdb,residential


In [279]:
split_df = no_rank.tenderer_name.apply(lambda x: re.sub(r' +', ' ', x).strip()).apply(lambda x: x.split(' '))
err_idx = split_df.index
split_df

2163    [3A, Master, Contract, Services, Pte, Ltd, &, ...
2241                [12, Wee, Hur, Development, Pte, Ltd]
Name: tenderer_name, dtype: object

In [280]:
ranking = split_df.apply(lambda x: x[0])
names = split_df.apply(lambda x: ' '.join(x[1:]) if len(x)>1 else np.nan)
names

2163    Master Contract Services Pte Ltd & Keong \nHon...
2241                          Wee Hur Development Pte Ltd
Name: tenderer_name, dtype: object

In [281]:
final_df.tenderer_rank[err_idx] = ranking
final_df.tenderer_name[err_idx] = names
final_df.isna().sum()

tenderer_rank    0
tenderer_name    0
tender_price     2
land_parcel      0
source           0
devt_type        0
dtype: int64

### deal with 3rd col

In [282]:
no_price = final_df[final_df.tender_price.isna()]
no_price

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
2319,8,Treasure View Pte Ltd and Raimon Land Developm...,NaN,Sengkang E20,hdb,residential
2347,3,CNQC Realty (Treasure) Investment Pte. Ltd. ...,NaN,Sembawang E3b,hdb,residential


In [283]:
split_df = no_price.tenderer_name.apply(lambda x: re.sub(r' +', ' ', x).strip()).apply(lambda x: x.split(' '))
err_idx = split_df.index
prices = split_df.apply(lambda x: x[-1])
names = split_df.apply(lambda x: ' '.join(x[:-1]) if len(x)>1 else np.nan)
err_idx

Int64Index([2319, 2347], dtype='int64')

In [284]:
final_df.tender_price[err_idx] = prices
final_df.tenderer_name[err_idx] = names
final_df.isna().sum()

tenderer_rank    0
tenderer_name    0
tender_price     0
land_parcel      0
source           0
devt_type        0
dtype: int64

In [285]:
no_name2 = final_df[final_df.tenderer_name.isna()]

In [286]:
split_df = no_name2.tender_price.apply(lambda x: re.sub(r' +', ' ', x).strip()).apply(lambda x: x.split(' '))
err_idx = split_df.index
prices = split_df.apply(lambda x: x[-1])
names = split_df.apply(lambda x: ' '.join(x[:-1]) if len(x)>1 else np.nan)
err_idx

Int64Index([], dtype='int64')

In [287]:
final_df.tender_price[err_idx] = prices
final_df.tenderer_name[err_idx] = names
final_df

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type
0,1,Hiap Hoe Holdings Pte ltd,"38,800,000.00",Hougang N5NC,hdb,mixed
1,2,Far East Organization Centre Pte Ltd,"34,390,000.00",Hougang N5NC,hdb,mixed
2,3,United Industrial Corporation \nLtd/Shenton Ho...,"26,888,000.00",Hougang N5NC,hdb,mixed
3,4,Wing Tai Land Pte Ltd,"18,000,000.00",Hougang N5NC,hdb,mixed
4,1,Golden Development Pte Ltd,"70,180,000.00",Hougang N9NC,hdb,mixed
...,...,...,...,...,...,...
2414,9,JBE (S) Pte. Ltd.,"211,000,000.00",Bukit Batok E11,hdb,residential
2415,1,CDL Zenith Pte. Ltd.,"336,068,000.00",Bukit Batok E12,hdb,residential
2416,2,Sim Lian Land Pte Ltd & Sim Lian Development \...,"335,508,000.00",Bukit Batok E12,hdb,residential
2417,3,Intrepid Investments Pte. Ltd. and TID \nResid...,"332,526,222.00",Bukit Batok E12,hdb,residential


In [291]:
err = final_df[final_df.tenderer_rank.apply(lambda x: x.split(' ')).str.len()>1]
err_idx = err.index

In [295]:
split_df = err.tenderer_rank.apply(lambda x: re.sub(r' +', ' ', x).strip()).apply(lambda x: x.split(' '))
ranking = split_df.apply(lambda x: x[0])
final_df.tenderer_rank[err_idx] = ranking
final_df.tenderer_rank[err_idx]

172      10
186      10
187      11
200      10
201     11a
202     11b
211      20
222      11
435      10
661      10
676      10
715      10
720      15
730      25
734      29
735      30
762      15
948      11
1028     3A
1117     11
1181     10
1187     5B
1249     10
1328    10a
1329    10b
1350     11
1366     11
1371     16
Name: tenderer_rank, dtype: object

In [296]:
final_df[final_df.tenderer_rank.apply(lambda x: x.split(' ')).str.len()>1]

,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type


In [297]:
def extract_num(string: str, type: str = 'all', decimal: bool = False, ignore_sep: str = None, keep: str = None):
    # 'type' means all numbers or just num between whitespaces by specifying type='between_spaces'
    # 'ignore_sep' can be 'any' to ignore all sep, or specify a sep like ',', then func won't treat ',' as a separator
    # 'keep' allows the func to keep all matched numbers or selected ones
    import re
    import itertools

    # if the input is already int or float, return itself: input=1234 -> output=1234
    if isinstance(string, int) or isinstance(string, float):
        num = string
        return num

    else:
        string = str(string)
        # # remove all spaces from string
        # string = ''.join(string.split(' '))
        try:
            # if the string can directly be converted to number, do so (e.g. input='1234' -> output=1234.0)
            num = float(string)
            return num

        except:
            pattern = r"\d+"  # find all numbers, any digits (ignore decimal number: input='$12.3' -> output=['12','3']
            if decimal:
                pattern = r"\d*\.?\d+"  # also match decimal numbers: input='$12.3' -> output='12.3'
            if type == 'between_spaces':
                pattern = r"\b" + pattern + r"\b"
                # match numbers in between white spaces
                # input='is $10.5 per box' -> output=None; input='is 10.5 dollars per box' -> output='10.5'
            num_list = re.findall(pattern, string)

            if ignore_sep:
                if ignore_sep == 'any':  # ignore any separator between numbers
                    # input='123a456,789.654' -> output='123456789654'
                    if len(num_list) >= 1:
                        num = "".join(num_list)
                        return float(num)
                    else:
                        return np.nan
                else:
                    # ignore specified separator
                    # input='$1,234,567.05' -> output ignore ',' & decimal is T='1234567.05'
                    # output without ignoring & decimal is T=['1','234','567.05']
                    string = string.replace(ignore_sep, "")
                    num_list = re.findall(pattern, string)
            num_list = [float(num) for num in num_list]  # convert all matched str item to float, stored in list

            if keep:  # to specify certain numbers to keep by index, e.g. num_list=[5, 6, 7], keep=1 -> output=[5]
                strip = [i.split(",") for i in keep.split("-")]
                # for now only support ",", for "-" will amend this later
                keep_idx = list(set([int(i) for i in list(itertools.chain.from_iterable(strip))]))
                if len(num_list) > len(keep_idx):  # if not keeping all items, raise a msg to notify
                    print(f"{len(num_list)} numbers detected")
                num_list = [num_list[i - 1] for i in keep_idx if 0 <= i - 1 < len(num_list)]

                if len(num_list) > 0:
                    return num_list[0] if len(num_list) == 1 else num_list
                else:
                    return np.nan

            if len(num_list) == 1:
                return num_list[0]  # if the result num_list has only 1 value, output the value as float
            elif len(num_list) > 1:
                return num_list  # otherwise output the whole num_list
            else:
                return np.nan # otherwise output NA

In [323]:
extract_num('23,000,000.00', decimal=True, ignore_sep=',')
final_df["tender_price_num"] = final_df.tender_price.apply(extract_num, decimal=True, ignore_sep=',')
final_df[final_df.tender_price_num.str.len()>1].tender_price_num = final_df[final_df.tender_price_num.str.len()>1].tender_price_num.values[0][0]
final_df[final_df.tender_price_num.str.len()>1]

E:\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,tenderer_rank,tenderer_name,tender_price,land_parcel,source,devt_type,tender_price_num
694,2,Far East Square Pte. Ltd.,"129,100,000 00",Bukit Panjang S10a,hdb,residential,"[129100000.0, 0.0]"


In [332]:
final_df.iloc[589, 6] = np.nan

In [ ]:
final_df.rename(columns={"tender_price": "tender_price_text", "tender_price_num": "tender_price"}, inplace=True)

In [336]:
final_df

,tenderer_rank,tenderer_name,tender_price_text,land_parcel,source,devt_type,tender_price
0,1,Hiap Hoe Holdings Pte ltd,"38,800,000.00",Hougang N5NC,hdb,mixed,38800000.0
1,2,Far East Organization Centre Pte Ltd,"34,390,000.00",Hougang N5NC,hdb,mixed,34390000.0
2,3,United Industrial Corporation \nLtd/Shenton Holdings Pte Ltd,"26,888,000.00",Hougang N5NC,hdb,mixed,26888000.0
3,4,Wing Tai Land Pte Ltd,"18,000,000.00",Hougang N5NC,hdb,mixed,18000000.0
4,1,Golden Development Pte Ltd,"70,180,000.00",Hougang N9NC,hdb,mixed,70180000.0
...,...,...,...,...,...,...,...
2414,9,JBE (S) Pte. Ltd.,"211,000,000.00",Bukit Batok E11,hdb,residential,211000000.0
2415,1,CDL Zenith Pte. Ltd.,"336,068,000.00",Bukit Batok E12,hdb,residential,336068000.0
2416,2,Sim Lian Land Pte Ltd & Sim Lian Development \nPte Ltd,"335,508,000.00",Bukit Batok E12,hdb,residential,335508000.0
2417,3,Intrepid Investments Pte. Ltd. and TID \nResidential Pte. Ltd.,"332,526,222.00",Bukit Batok E12,hdb,residential,332526222.0


In [337]:
final_df[["land_parcel", "tenderer_rank", "tenderer_name", "tender_price", "tender_price_text"]].to_csv("hdb_tenderer_details.csv", index=False, header=True)